<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy.linalg import inv
import random

In [2]:
#откроем файл с данными
df = pd.read_csv('/datasets/insurance.csv')
#изучим данные
display(df.head())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Из исходных данных мы знаем, что:    
 - Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.    
 - Целевой признак: количество страховых выплат клиенту за последние 5 лет.    
Для определения типа задачи(категориальная или количественная), определим, какие значения находятся в столбце "Страховые выплаты".

In [3]:
#определим уникальные значения столбца и их количество
df['Страховые выплаты'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

Как мы видим, числовые значения представлены в диапазоне, а значит данные - количественные. Т.к. целевой признак количественный, мы будем решать задачу регрессии.

In [4]:
#изучим данные воспользовавшись унифицированной функцией
def uni_function(function):
    display(function.head())
    display(function.info())
    display(function.isna().mean()) #доля пропущенных значений
    display(function.describe())
    display(function.duplicated().sum())

In [5]:
uni_function(df)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


153

Посде изучения данных мы видим:    
 1. В исходной таблице отсутствуют пропущенные значения, однако было обнаружено 153 дубликата.
 2. Имя переменных указаны не по правилам "хорошего стиля". Имена на русском языке, без использования "змеиного регистра" и с использованием заглавных букв. 
 
Для дальнейшей работы устраним эти недочеты.

Переименуем названия столбцов в соответсвии с правилами:
 - Пол - gender;
 - Возраст - age;
 - Зарплата - salary;
 - Члены семьи - family_members;
 - Страховые выплаты - insurance_payments.

In [6]:
#переименуем столбцы
df = df.rename(columns={'Пол': 'gender', 
                        'Возраст': 'age', 
                        'Зарплата': 'salary', 
                        'Члены семьи': 'family_members', 
                        'Страховые выплаты': 'insurance_payments'})
display(df.head())

,gender,age,salary,family_members,insurance_payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
#удалим явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              4847 non-null   int64  
 1   age                 4847 non-null   float64
 2   salary              4847 non-null   float64
 3   family_members      4847 non-null   int64  
 4   insurance_payments  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


Данные готовы к работе.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится

**Обоснование:**    
По заданию нам необходимо умножить исходную матрицу(признаки) на обратимую матрицу(единичну матрицу).    
Исходная матрица Х имеет размеры mxn(5000х4), единичная матрица P является квадратной - nxn(4x4).    
Запишем значение весов опираясь на новые данные:    

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Запишем формулу предсказаний:

$$
a' = XP ((XP)^T XP)^{-1} (XP)^T y
$$

Для решения даннной задачи нам необходимо помнить следующие свойства:    
1. Если любую матрицу A умножить на единичную (или наоборот), получится эта же матрица A
$$
AE = EA = A
$$
2. Обратная для квадратной матрицы A — матрица A с верхним индексом -1, произведение которой на А равно единичной матрице.
$$
AA^{-1} = A^{-1}A = E
$$
3. Сочетательное свойство матриц 
$$ (AB)C = A(BC) $$

Свойства степеней

$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
    
Раскроем скобки в формуле предсказаний учитывая транспонирование:
$$
a' = XP (P^T X^T XP)^{-1} P^{T} X^{T} y
$$
Применяя свойства матриц, преобразуем выражение внутри скобок:
$$
a' = XP (P^T(X^T X) P)^{-1} P^{T} X^{T}y = X P P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
Произведения $PP^{-1}$ и $(P^T)^{-1} P^T$ дают единичные матрицы, а значит мы можем их убрать из уравнения.    
Запишем получившееся уравнение:
$$
a' = X (X^T X)^{-1} X^T y
$$
Выражение $$X (X^T X)^{-1} X^T y = w$$. Запишем итоговое выражение:
$$
a' = Xw
$$
Как мы видим, исходная запись предсказания совпала с полученной, а значит качество линейной регрессии не изменится.
$$
a' = a
$$

## Алгоритм преобразования

**Алгоритм**    
Алгоритмом преобразования будет являться домножение признаков X на случайную обратимую матрицу Р размером 4х4. Создадим модель линейной регрессии до преобразования и посчитаем метрику R2. Далее проделаем тоже самое с домноженной на обратимую матрицу матрицей и снова посчитаем метрику R2. По итогу метрики R2 должны быть равны.

**Обоснование**

Как было показано в предыдущем пункте, домножение признаков на какую-либо обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны.

## Проверка алгоритма

In [9]:
features = df.drop(['insurance_payments'], axis = 1) #признаки
target = df['insurance_payments'] #целевой признак

In [10]:
#обучим модель
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print(f'Метрика R2 для признаков в исходном состоянии равняется {r2}')

Метрика R2 для признаков в исходном состоянии равняется 0.4302010044852066


In [11]:
#Сгенерируем случайную матрицу размером 4х4 функцией numpy.random.normal()
r = np.random.RandomState(42)
random_matrix = r.normal(size = (4, 4))
display(random_matrix)

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986],
       [-0.23415337, -0.23413696,  1.57921282,  0.76743473],
       [-0.46947439,  0.54256004, -0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024, -1.72491783, -0.56228753]])

In [12]:
#найдем обратную матрицу
invert_matrix = inv(np.matrix(random_matrix))
display(invert_matrix)

matrix([[-0.33124857, -1.0333483 , -2.21070923, -0.47651024],
        [ 0.12864778, -0.47781825,  0.15271687, -0.43017975],
        [-0.51338817,  0.33588835, -0.8347692 , -0.24072429],
        [ 0.99462192,  0.15079316,  1.08985145,  0.21872535]])

In [13]:
#домножим нашу исходную матрицу на обратную
new_matrix = features@invert_matrix
display(new_matrix.head())

,0,1,2,3
0,-25458.115200,16639.589185,-41399.411722,-11957.820064
1,-19501.837964,12741.928552,-31713.114724,-9167.092659
2,-10777.420742,7039.798675,-17525.724384,-5067.685356
3,-21403.595758,13996.811706,-34804.488829,-10046.799323
4,-13396.160295,8752.273743,-21785.410723,-6295.425578


In [14]:
#произведем моделирование на новых данных
model_2 = LinearRegression()
model_2.fit(new_matrix, target)
predictions_2 = model_2.predict(new_matrix)
r2_new = r2_score(target, predictions_2)
print(f'Метрика R2 для признаков в исходном состоянии равняется {r2_new}')

Метрика R2 для признаков в исходном состоянии равняется 0.43020100448520615


In [15]:
#сравним два массива данных
np.isclose(r2, r2_new)

True

**Вывод**    
При умножение матрицы признаков на обратимую матрицу качество Линейной Регрессии не изменяется.